In [ ]:
import numpy as np
import pandas as pd
import json
from collections import defaultdict
from pathlib import Path
from wsgi import read_ak_data, read_info, read_ak_list, read_blocks

In [ ]:
poll_name = "koma90"
export_dir = Path("export") / poll_name

In [ ]:
data

In [ ]:
data = read_ak_data(poll_name)
ak_list = read_ak_list(data)
block_dict = read_blocks(data, default={})

In [ ]:
name_lst = []
for result_json in export_dir.glob("*.json"):
    with result_json.open("r") as ff:
        result_dict = json.load(ff)
    result_dict["info"]["filename"] = result_json.name
    result_dict["preferences"].append(
        {
          "ak_id": "ak40",
          "required": False,
          "preference_score": 1
        }
    )
    name_lst.append(
        [
            f"{result_dict['info']['name']} ({result_dict['info']['uni']})",
            result_json.name
        ]
    )
with open("/tmp/tmp.txt", "w") as ff:
    json.dump(list(sorted(name_lst)), ff, indent=2, ensure_ascii=False)

In [ ]:
for idx, ak in enumerate(data["aks"]):
    ak["id"] = f"ak{idx}"
    if "required_time_constraints" in ak:
        ak["time_constraints"] = ak["required_time_constraints"]
        del ak["required_time_constraints"]
    elif not "time_constraints" in ak:
        ak["time_constraints"] = []

    if "duration" not in ak:
        if "duration" in ak["info"]:
            ak["duration"] = int(ak["info"]["duration"])
        else:
            ak["duration"] = 2
    
    if isinstance(ak["duration"], str):
        print(ak)

    if "required_room_constraints" in ak:
        ak["room_constraints"] = ak["required_room_constraints"]
        del ak["required_room_constraints"]
    elif not "room_constraints" in ak:
        ak["room_constraints"] = []

    ak["properties"] = {}

In [ ]:
participants = []
for result_json in export_dir.glob("*.json"):
    with result_json.open("r") as ff:
        result_dict = json.load(ff)
    result_dict["id"] = result_json.stem
    if "required_time_constraints" in result_dict.keys():
        result_dict["time_constraints"] = result_dict["required_time_constraints"]
        del result_dict["required_time_constraints"]
    if not "time_constraints" in result_dict.keys():
        result_dict["room_constraints"] = []
    participants.append(result_dict)
with open("participants.json", "w") as ff:
    json.dump(participants, ff, ensure_ascii=False, indent=2)

In [ ]:
data["participants"] = participants

with open("export.json", "w") as ff:
    json.dump(data, ff, ensure_ascii=False, indent=2)

In [ ]:


if (data := read_ak_data(poll_name)) is not None:
    
    ak_pref_dict = defaultdict(lambda: defaultdict(int))
    time_constraint_dict = defaultdict(lambda: defaultdict(int))
    block_lst = []
    for day, day_entries in block_dict.items():
        block_lst.extend(
            [f"{day[:2]} {slot_name}" for block_idx, slot_name in day_entries]
        )
    
    ak_pref_dict = {k: dict(v) for k, v in ak_pref_dict.items()}
    for k in ak_pref_dict:
        ak_pref_dict[k]["time_constraints"] = dict(time_constraint_dict[k])